In [ ]:
from torch import nn
from torchvision.transforms import Compose,ToTensor,Normalize
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader
from torch.optim import Adam
import torch.nn.functional as F
import numpy as np
import torch, os

In [ ]:
BATCH_SIZE = 16
LEARNING_RATE = 0.005

def get_dataloader(train=True):
    transform_fn = Compose([ ToTensor(), 
                             Normalize(mean=(0.1307,), std=(0.3081,)) ]) #转换成Tensor并且标准化
    dataset = MNIST(root='./MNISTdata', download=False, train=train, transform=transform_fn) #实例化数据集
    data_loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True) #实例化数据迭代器
    return data_loader

class MnistModel(nn.Module):
    def __init__(self):
        super(MnistModel,self).__init__()
        self.fc1 = nn.Linear(1*28*28, 28) #第一个全连接层输入1层*28列*28行，输出28个
        self.fc2 = nn.Linear(28, 10) #第二层输入28个，输出10个
    def forward(self,inputs):
        x = inputs.view([BATCH_SIZE, -1]) #形状修改，把
        x = self.fc1(x) #全连接操作
        x = F.relu(x) #激活函数处理
        out = self.fc2(x) #全连接操作
        return out 

device = torch.device('cuda')
model = MnistModel().to(device) #实例化模型
criterion = nn.CrossEntropyLoss() #实例化损失函数
optimizer = Adam(model.parameters(), lr=LEARNING_RATE) #实例化优化器，传入模模型参数和型学习率

def train():
    data_loader = get_dataloader(train=True)#实例化数据迭代器
    for idx,(inputs,target) in enumerate(data_loader):
        inputs,target = inputs.to(device),target.to(device)
        optimizer.step() #优化器重置
        optimizer.zero_grad() #在优化器中梯度置零
        output = model(inputs) #调用模型得到预测值
        loss = criterion(output, target) #在损失函数中根据得到损失
        loss.backward() #损失的反向传播
        if idx%300==0:
            print('序号：'+str(idx), 'LOSS：'+str(loss.item()) )
        if idx%300==0:
            torch.save(model.state_dict(), './model/mnist_model.pt')
            torch.save(optimizer.state_dict(), './model/mnist_optimizer.pt')
            #模型的保存

In [ ]:
if os.path.exists('./model/mnist_optimizer.pt') and os.path.exists('./model/mnist_model.pt'):
    model.load_state_dict(torch.load('./model/mnist_model.pt'))
    optimizer.load_state_dict(torch.load('./model/mnist_optimizer.pt'))
    #模型的加载

In [ ]:
train()

# ======================================================

In [ ]:
def test():
    model.eval()
    loss_list = []; acc_list = []
    test_dataloader = get_dataloader(train=False)
    #导入模型，确定评估模式
    for idx,(inputs, target) in enumerate(test_dataloader):
        inputs,target = inputs.to(device),target.to(device)
        with torch.no_grad():
            output = model(inputs) #运算结果
            cur_loss = criterion(output, target) #计算loss
            loss_list.append(cur_loss) #把loss值添加到列表
            #output:[batch_size,10]  target:[batch.size]
            pred = output.max(dim=-1)[1] #取出预测值的最大值的未知
            cur_acc = pred.eq(target).float().mean() #判断与目标值是否相等，把布尔型转化为整形，求均值
            acc_list.append(cur_acc) #把准确率加入列表
        if idx%20==0:
            print('序号：'+str(idx), '准确率：'+str(cur_acc.item()) )
    # 正向运算并保存结果到列表
    loss_list = torch.tensor(loss_list, device=device)
    acc_list = torch.tensor(acc_list, device=device) #把列表转化为tensor便于求均值
    print('准确率：',acc_list.mean().item(), 'LOSS：',loss_list.mean().item())
    # 计算并输出准确率
test()
#计算评估准确率